In [1]:
import pandas as pd
pd.set_option('display.max_rows', 60)
pd.set_option('display.max_columns', None) 
import numpy as np
import re
from collections import defaultdict

In [2]:
yahoo_players = pd.read_csv('Raw/players_yahooleague_raw_14to25.csv')
yahoo_players.head()
yahoo_players = yahoo_players.drop(columns='Unnamed: 0')
yahoo_players.head()

master_ids = pd.read_csv('Raw/player_keys_raw.csv')
master_ids.head()
master_ids = master_ids.drop(columns='Unnamed: 0')
master_ids.head()

master_raw = master_ids.copy()

In [3]:
master_raw['position'].value_counts()

position
WR    2038
LB    1779
RB    1587
CB    1268
S     1083
DT     969
DE     957
TE     948
QB     666
PK     276
PN     215
DB     146
DL     136
OT      53
XX      16
C        6
T        1
?        1
Name: count, dtype: int64

In [4]:
master_raw[master_raw['name'] == 'Peyton Hillis']

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season
6857,9272,a34a5944-33c4-4965-b404-6772f4b7f1ab,9506.0,00-0026367,4541.0,660.0,1980.0,11461.0,900035.0,NaN,523458.0,HillPe00,peyton-hillis-1,5866.0,4965.0,NaN,9004.0,267714.0,11747.0,NaN,Peyton Hillis,peyton hillis,RB,FA,1986-01-21,35.3,2008.0,7.0,20.0,227.0,thepeytonhillis,74.0,250.0,Arkansas,2025


In [5]:
master_raw = master_raw.loc[master_raw['position'].isin(['QB', 'RB', 'TE', 'WR', 'PK', 'XX'])]
master_raw[master_raw['name'] == 'Jacoby Jones']

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season
7043,8772,ce4d7d70-307f-4093-bf54-432b8b405eb4,9253.0,00-0025460,3689.0,203.0,2495651.0,10517.0,8327.0,4674.0,1220258.0,JoneJa03,NaN,5229.0,4239.0,NaN,8327.0,329337.0,5100.0,NaN,Jacoby Jones,jacoby jones,WR,FA,1984-07-11,36.8,2007.0,3.0,9.0,73.0,TheRealJacoby12,76.0,215.0,Lane,2025


In [6]:
# Create name cleaning funciton
def clean_name(name):
    if pd.isna(name) or name is None:
        return name
    name = re.sub(r'\s+(Jr\.?|Sr\.?|III|II|IV|V)(\s|$|,)', '', str(name), flags=re.IGNORECASE).strip()
    name = name.replace("'", "")
    return name

In [7]:
yahoo_players['cleaned_name'] = yahoo_players['name.full'].apply(clean_name)

In [8]:
master_raw['cleaned_name'] = master_raw['name'].apply(clean_name)

In [9]:
yahoo_players['match_key'] = yahoo_players['cleaned_name'] + '_' + yahoo_players['player_id'].astype('Int64').astype(str)
master_raw['match_key'] = master_raw['cleaned_name'] + '_' + master_raw['stats_id'].astype('Int64').astype(str)

In [10]:
master_raw[master_raw['name'] == 'Jacoby Jones']

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key
7043,8772,ce4d7d70-307f-4093-bf54-432b8b405eb4,9253.0,00-0025460,3689.0,203.0,2495651.0,10517.0,8327.0,4674.0,1220258.0,JoneJa03,NaN,5229.0,4239.0,NaN,8327.0,329337.0,5100.0,NaN,Jacoby Jones,jacoby jones,WR,FA,1984-07-11,36.8,2007.0,3.0,9.0,73.0,TheRealJacoby12,76.0,215.0,Lane,2025,Jacoby Jones,Jacoby Jones_8327


In [11]:
master_raw[master_raw['match_key'].str.contains('_<NA>')]

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key
35,17065,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,Henry Parrish,henry parrish,RB,FA,NaN,NaN,2025.0,NaN,NaN,NaN,NaN,70.0,185.0,Mississippi,2025,Henry Parrish,Henry Parrish_<NA>
37,17067,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,John Emery,john emery,RB,FA,NaN,NaN,2025.0,NaN,NaN,NaN,NaN,72.0,220.0,LSU,2025,John Emery,John Emery_<NA>
59,17089,483872b0-2630-11f0-b7f1-df7c830143e7,27218.0,NaN,NaN,12460.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18559.0,NaN,NaN,NaN,0.0,26081.0,NaN,Antwane Wells,antwane wells,WR,NYG,2001-04-02,24.5,2025.0,NaN,NaN,NaN,NaN,72.0,205.0,Ole Miss,2025,Antwane Wells,Antwane Wells_<NA>
64,17094,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18456.0,NaN,NaN,NaN,0.0,NaN,NaN,Kyren Lacy,kyren lacy,XX,FA,NaN,NaN,2025.0,NaN,NaN,NaN,NaN,74.0,213.0,LSU,2025,Kyren Lacy,Kyren Lacy_<NA>
80,17110,NaN,NaN,NaN,NaN,11594.0,NaN,NaN,NaN,NaN,2965087.0,NaN,NaN,18541.0,NaN,NaN,NaN,0.0,NaN,NaN,Brant Kuithe,brant kuithe,TE,FA,NaN,NaN,2025.0,NaN,NaN,NaN,NaN,74.0,236.0,Utah,2025,Brant Kuithe,Brant Kuithe_<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12139,968,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sean Brewer,sean brewer,TE,CIN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025,Sean Brewer,Sean Brewer_<NA>
12140,972,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Doug Brien,doug brien,PK,NOS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025,Doug Brien,Doug Brien_<NA>
12141,978,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jeremy Brigham,jeremy brigham,TE,OAK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025,Jeremy Brigham,Jeremy Brigham_<NA>
12142,984,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vincent Brisby,vincent brisby,WR,FA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025,Vincent Brisby,Vincent Brisby_<NA>


In [12]:
print(f"Memory usage: {master_raw.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

Memory usage: 5.25 MB


In [13]:
# Rookies missing are on the yahoo list, will make a yahoo ID map and use to add to master list

yahoo_id_maps = yahoo_players.set_index('match_key')['player_id'].to_dict()


In [14]:
print(yahoo_id_maps['Jacoby Jones_8327'])

8327


In [15]:
master_raw['mapped_ids'] = master_raw['match_key'].map(yahoo_id_maps)
display(master_raw[master_raw['name'] == 'Ben Wooldridge'])
display(master_raw[master_raw['name'] == 'Travis Hunter'])


,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids
301,17338,0eba8690-2572-11f0-841c-6faaaac5712d,NaN,NaN,NaN,13065.0,NaN,4373895.0,NaN,NaN,2965115.0,NaN,NaN,18872.0,NaN,NaN,4373895.0,0.0,NaN,NaN,Ben Wooldridge,ben wooldridge,QB,FA,1999-09-27,26.1,2025.0,NaN,NaN,NaN,NaN,74.0,214.0,Louisiana,2025,Ben Wooldridge,Ben Wooldridge_4373895,NaN


,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids
44,17074,4a975d4c-d7cd-45ef-b258-db45187140eb,26034.0,00-0040718,160766.0,12530.0,58204.0,4685415.0,NaN,NaN,28909849.0,HuntTr00,NaN,18578.0,NaN,1774.0,41787.0,0.0,26056.0,NaN,Travis Hunter,travis hunter,WR,JAC,2003-05-18,22.4,2025.0,1.0,2.0,2.0,NaN,73.0,185.0,Colorado,2025,Travis Hunter,Travis Hunter_41787,NaN


In [16]:
master_raw[(master_raw['mapped_ids'].isna())]

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids
35,17065,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,Henry Parrish,henry parrish,RB,FA,NaN,NaN,2025.0,NaN,NaN,NaN,NaN,70.0,185.0,Mississippi,2025,Henry Parrish,Henry Parrish_<NA>,NaN
37,17067,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,John Emery,john emery,RB,FA,NaN,NaN,2025.0,NaN,NaN,NaN,NaN,72.0,220.0,LSU,2025,John Emery,John Emery_<NA>,NaN
44,17074,4a975d4c-d7cd-45ef-b258-db45187140eb,26034.0,00-0040718,160766.0,12530.0,58204.0,4685415.0,NaN,NaN,28909849.0,HuntTr00,NaN,18578.0,NaN,1774.0,41787.0,0.0,26056.0,NaN,Travis Hunter,travis hunter,WR,JAC,2003-05-18,22.4,2025.0,1.0,2.0,2.0,NaN,73.0,185.0,Colorado,2025,Travis Hunter,Travis Hunter_41787,NaN
59,17089,483872b0-2630-11f0-b7f1-df7c830143e7,27218.0,NaN,NaN,12460.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18559.0,NaN,NaN,NaN,0.0,26081.0,NaN,Antwane Wells,antwane wells,WR,NYG,2001-04-02,24.5,2025.0,NaN,NaN,NaN,NaN,72.0,205.0,Ole Miss,2025,Antwane Wells,Antwane Wells_<NA>,NaN
64,17094,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18456.0,NaN,NaN,NaN,0.0,NaN,NaN,Kyren Lacy,kyren lacy,XX,FA,NaN,NaN,2025.0,NaN,NaN,NaN,NaN,74.0,213.0,LSU,2025,Kyren Lacy,Kyren Lacy_<NA>,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12139,968,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sean Brewer,sean brewer,TE,CIN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025,Sean Brewer,Sean Brewer_<NA>,NaN
12140,972,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Doug Brien,doug brien,PK,NOS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025,Doug Brien,Doug Brien_<NA>,NaN
12141,978,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jeremy Brigham,jeremy brigham,TE,OAK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025,Jeremy Brigham,Jeremy Brigham_<NA>,NaN
12142,984,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vincent Brisby,vincent brisby,WR,FA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025,Vincent Brisby,Vincent Brisby_<NA>,NaN


In [17]:
# Count NaN in mapped_ids
print("NaN in mapped_ids:", master_raw['mapped_ids'].isna().sum())

# Check if mismatch should include NaN
print("Should show mismatches:", (~master_raw['mapped_ids'].isin(master_raw['stats_id'])).sum())

# Are the NaN values in mapped_ids also NaN in stats_id?
both_nan = master_raw['mapped_ids'].isna() & master_raw['stats_id'].isna()
print("Rows where BOTH are NaN:", both_nan.sum())

NaN in mapped_ids: 2601
Should show mismatches: 0
Rows where BOTH are NaN: 1093


In [18]:
master_raw[master_raw['name'] == 'Adam Vinatieri']

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids
9688,2842,9ecf8040-10f9-4a5c-92da-1b4d77bd6760,9004.0,00-0016919,226.0,120.0,2503471.0,1097.0,3727.0,2074.0,1392.0,vinatada01,NaN,395.0,1152.0,NaN,3727.0,23849.0,3258.0,NaN,Adam Vinatieri,adam vinatieri,PK,FA,1972-12-28,49.2,1996.0,NaN,NaN,NaN,aVinatieri4,72.0,212.0,South Dakota State,2025,Adam Vinatieri,Adam Vinatieri_3727,3727.0


In [19]:
# Exclude rows where BOTH are NaN (those are fine)
# Show rows where they don't match
master_mismatch = master_raw[
    (master_raw['mapped_ids'] != master_raw['stats_id']) & 
    ~(master_raw['mapped_ids'].isna() & master_raw['stats_id'].isna())
]
print(f"True mismatches: {len(master_mismatch)}")

True mismatches: 1508


In [20]:
master_mismatch.head()

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids
44,17074,4a975d4c-d7cd-45ef-b258-db45187140eb,26034.0,00-0040718,160766.0,12530.0,58204.0,4685415.0,NaN,NaN,28909849.0,HuntTr00,NaN,18578.0,NaN,1774.0,41787.0,0.0,26056.0,NaN,Travis Hunter,travis hunter,WR,JAC,2003-05-18,22.4,2025.0,1.0,2.0,2.0,NaN,73.0,185.0,Colorado,2025,Travis Hunter,Travis Hunter_41787,NaN
190,17220,80dce1ba-c997-47e6-a37e-e92fefcbdc1b,NaN,00-0040200,143934.0,12713.0,58387.0,4569923.0,NaN,NaN,26699665.0,BorrAn00,NaN,18743.0,NaN,NaN,41980.0,0.0,26106.0,NaN,Andres Borregales,andres borregales,PK,NEP,2002-01-02,23.8,2025.0,6.0,6.0,182.0,NaN,71.0,202.0,Miami (FL),2025,Andres Borregales,Andres Borregales_41980,NaN
294,17329,826620a0-249a-11f0-9302-cb167d81eff7,23170.0,00-0040332,123119.0,12479.0,58768.0,4690972.0,NaN,NaN,3168195.0,JackJa05,NaN,18527.0,NaN,NaN,4690972.0,0.0,25901.0,NaN,Ja'Quinden Jackson,jaquinden jackson,RB,JAC,2001-09-12,24.1,2025.0,NaN,NaN,NaN,NaN,74.0,233.0,Arkansas,2025,JaQuinden Jackson,JaQuinden Jackson_4690972,NaN
301,17338,0eba8690-2572-11f0-841c-6faaaac5712d,NaN,NaN,NaN,13065.0,NaN,4373895.0,NaN,NaN,2965115.0,NaN,NaN,18872.0,NaN,NaN,4373895.0,0.0,NaN,NaN,Ben Wooldridge,ben wooldridge,QB,FA,1999-09-27,26.1,2025.0,NaN,NaN,NaN,NaN,74.0,214.0,Louisiana,2025,Ben Wooldridge,Ben Wooldridge_4373895,NaN
358,16590,ec7b8cf0-0bb9-11ef-bb45-99966e93bb71,23002.0,NaN,NaN,12259.0,NaN,NaN,NaN,NaN,3123935.0,NaN,NaN,16999.0,NaN,NaN,41559.0,0.0,NaN,NaN,Taulia Tagovailoa,taulia tagovailoa,QB,FA,2000-02-15,25.7,2024.0,NaN,NaN,NaN,NaN,71.0,200.0,Maryland,2025,Taulia Tagovailoa,Taulia Tagovailoa_41559,NaN


In [21]:
master_mismatch[master_mismatch['stats_id'] > 101000]

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids
294,17329,826620a0-249a-11f0-9302-cb167d81eff7,23170.0,00-0040332,123119.0,12479.0,58768.0,4690972.0,NaN,NaN,3168195.0,JackJa05,NaN,18527.0,NaN,NaN,4690972.0,0.0,25901.0,NaN,Ja'Quinden Jackson,jaquinden jackson,RB,JAC,2001-09-12,24.1,2025.0,NaN,NaN,NaN,NaN,74.0,233.0,Arkansas,2025,JaQuinden Jackson,JaQuinden Jackson_4690972,NaN
301,17338,0eba8690-2572-11f0-841c-6faaaac5712d,NaN,NaN,NaN,13065.0,NaN,4373895.0,NaN,NaN,2965115.0,NaN,NaN,18872.0,NaN,NaN,4373895.0,0.0,NaN,NaN,Ben Wooldridge,ben wooldridge,QB,FA,1999-09-27,26.1,2025.0,NaN,NaN,NaN,NaN,74.0,214.0,Louisiana,2025,Ben Wooldridge,Ben Wooldridge_4373895,NaN
666,16964,9f2c6ea0-0570-11ef-af69-dfa5fe884ba6,NaN,00-0039614,141559.0,11897.0,57742.0,4698301.0,NaN,NaN,26717777.0,JackJe01,NaN,18104.0,NaN,NaN,4698301.0,0.0,NaN,1274459.0,Jermaine Jackson,jermaine jackson,WR,FA,2000-06-02,25.4,2024.0,NaN,NaN,NaN,NaN,67.0,170.0,Idaho,2025,Jermaine Jackson,Jermaine Jackson_4698301,NaN


These three player shave no stats, will drop

In [22]:
master_mismatch = master_mismatch[master_mismatch['stats_id'] < 101000]
master_mismatch[master_mismatch['stats_id'] > 101000]

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids


In [23]:
master_mismatch.loc[(master_mismatch['stats_id'].isna()), 'stats_id'] = master_mismatch.loc[(master_mismatch['stats_id'].isna()), 'mapped_ids']

In [24]:
master_mismatch[master_mismatch['stats_id'].isna()]

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids


In [25]:
master_mismatch[master_mismatch['stats_id'].isna()]

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids


In [26]:
master_mismatch.head()

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids
44,17074,4a975d4c-d7cd-45ef-b258-db45187140eb,26034.0,00-0040718,160766.0,12530.0,58204.0,4685415.0,NaN,NaN,28909849.0,HuntTr00,NaN,18578.0,NaN,1774.0,41787.0,0.0,26056.0,NaN,Travis Hunter,travis hunter,WR,JAC,2003-05-18,22.4,2025.0,1.0,2.0,2.0,NaN,73.0,185.0,Colorado,2025,Travis Hunter,Travis Hunter_41787,NaN
190,17220,80dce1ba-c997-47e6-a37e-e92fefcbdc1b,NaN,00-0040200,143934.0,12713.0,58387.0,4569923.0,NaN,NaN,26699665.0,BorrAn00,NaN,18743.0,NaN,NaN,41980.0,0.0,26106.0,NaN,Andres Borregales,andres borregales,PK,NEP,2002-01-02,23.8,2025.0,6.0,6.0,182.0,NaN,71.0,202.0,Miami (FL),2025,Andres Borregales,Andres Borregales_41980,NaN
358,16590,ec7b8cf0-0bb9-11ef-bb45-99966e93bb71,23002.0,NaN,NaN,12259.0,NaN,NaN,NaN,NaN,3123935.0,NaN,NaN,16999.0,NaN,NaN,41559.0,0.0,NaN,NaN,Taulia Tagovailoa,taulia tagovailoa,QB,FA,2000-02-15,25.7,2024.0,NaN,NaN,NaN,NaN,71.0,200.0,Maryland,2025,Taulia Tagovailoa,Taulia Tagovailoa_41559,NaN
585,16818,8e6fc170-07d2-11ef-95d0-7db2ea91136a,NaN,NaN,NaN,12188.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41477.0,0.0,NaN,NaN,Justin Strong,justin strong,RB,FA,NaN,NaN,2024.0,NaN,NaN,NaN,NaN,72.0,205.0,Southern Illinois,2025,Justin Strong,Justin Strong_41477,NaN
600,16833,fffee080-04ff-11ef-8a9a-5f3b29e3b4c8,26405.0,00-0039541,124169.0,11886.0,57660.0,4429153.0,41163.0,NaN,3171022.0,GrifLi00,NaN,17973.0,NaN,NaN,41163.0,0.0,NaN,1231735.0,Lideatrick Griffin,lideatrick griffin,WR,FA,2001-01-23,24.7,2024.0,NaN,NaN,NaN,NaN,70.0,181.0,Mississippi State,2025,Lideatrick Griffin,Lideatrick Griffin_41163,NaN


In [27]:
master_mismatch.loc[master_mismatch['stats_id'].duplicated(keep=False)]

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids


In [28]:
master_mismatch.loc[master_mismatch['stats_id'] < 3000].head()

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids
9848,1032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BantBr00,bradford-banta-1,837.0,NaN,NaN,2732.0,0.0,NaN,NaN,Bradford Banta,bradford banta,TE,FA,1970-12-14,50.4,1994.0,4.0,3.0,106.0,NaN,78.0,253.0,Southern California,2025,Bradford Banta,Bradford Banta_2732,NaN
9852,1076,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,briendou01,doug-brien-1,849.0,NaN,NaN,2935.0,0.0,NaN,NaN,Doug Brien,doug brien,PK,FA,1970-11-24,50.5,1994.0,3.0,20.0,85.0,NaN,NaN,185.0,California,2025,Doug Brien,Doug Brien_2935,NaN
9854,1095,NaN,NaN,00-0002099,NaN,NaN,NaN,658.0,NaN,322.0,1294.0,BrucIs00,isaac-bruce-1,626.0,414.0,NaN,2914.0,0.0,NaN,NaN,Isaac Bruce,isaac bruce,WR,FA,1972-11-10,48.5,1994.0,2.0,4.0,33.0,NaN,72.0,188.0,Memphis,2025,Isaac Bruce,Isaac Bruce_2914,NaN
9858,1183,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1065.0,DilfTr00,trent-dilfer-1,666.0,552.0,NaN,2900.0,0.0,NaN,NaN,Trent Dilfer,trent dilfer,QB,FA,1972-03-13,49.2,1994.0,1.0,6.0,6.0,DilfersDimes,76.0,234.0,Fresno State,2025,Trent Dilfer,Trent Dilfer_2900,NaN
9861,1213,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FaulMa00,marshall-faulk-1,292.0,368.0,NaN,2728.0,0.0,NaN,NaN,Marshall Faulk,marshall faulk,RB,FA,1973-02-26,48.2,1994.0,1.0,2.0,2.0,marshallfaulk,70.0,211.0,NaN,2025,Marshall Faulk,Marshall Faulk_2728,NaN


In [29]:
master_mismatch = master_mismatch.drop(labels=master_mismatch.loc[master_mismatch['stats_id'] < 3726].index, axis=0)
master_mismatch['stats_id'].min()

np.float64(3731.0)

In [30]:
master_mismatch[~master_mismatch['mapped_ids'].isin(master_mismatch['stats_id'])]

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids
44,17074,4a975d4c-d7cd-45ef-b258-db45187140eb,26034.0,00-0040718,160766.0,12530.0,58204.0,4685415.0,NaN,NaN,28909849.0,HuntTr00,NaN,18578.0,NaN,1774.0,41787.0,0.0,26056.0,NaN,Travis Hunter,travis hunter,WR,JAC,2003-05-18,22.4,2025.0,1.0,2.0,2.0,NaN,73.0,185.0,Colorado,2025,Travis Hunter,Travis Hunter_41787,NaN
190,17220,80dce1ba-c997-47e6-a37e-e92fefcbdc1b,NaN,00-0040200,143934.0,12713.0,58387.0,4569923.0,NaN,NaN,26699665.0,BorrAn00,NaN,18743.0,NaN,NaN,41980.0,0.0,26106.0,NaN,Andres Borregales,andres borregales,PK,NEP,2002-01-02,23.8,2025.0,6.0,6.0,182.0,NaN,71.0,202.0,Miami (FL),2025,Andres Borregales,Andres Borregales_41980,NaN
358,16590,ec7b8cf0-0bb9-11ef-bb45-99966e93bb71,23002.0,NaN,NaN,12259.0,NaN,NaN,NaN,NaN,3123935.0,NaN,NaN,16999.0,NaN,NaN,41559.0,0.0,NaN,NaN,Taulia Tagovailoa,taulia tagovailoa,QB,FA,2000-02-15,25.7,2024.0,NaN,NaN,NaN,NaN,71.0,200.0,Maryland,2025,Taulia Tagovailoa,Taulia Tagovailoa_41559,NaN
585,16818,8e6fc170-07d2-11ef-95d0-7db2ea91136a,NaN,NaN,NaN,12188.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41477.0,0.0,NaN,NaN,Justin Strong,justin strong,RB,FA,NaN,NaN,2024.0,NaN,NaN,NaN,NaN,72.0,205.0,Southern Illinois,2025,Justin Strong,Justin Strong_41477,NaN
600,16833,fffee080-04ff-11ef-8a9a-5f3b29e3b4c8,26405.0,00-0039541,124169.0,11886.0,57660.0,4429153.0,41163.0,NaN,3171022.0,GrifLi00,NaN,17973.0,NaN,NaN,41163.0,0.0,NaN,1231735.0,Lideatrick Griffin,lideatrick griffin,WR,FA,2001-01-23,24.7,2024.0,NaN,NaN,NaN,NaN,70.0,181.0,Mississippi State,2025,Lideatrick Griffin,Lideatrick Griffin_41163,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12034,9793,20af511b-03b7-4905-bc41-158f286b99c6,NaN,NaN,NaN,2864.0,2553937.0,12899.0,29138.0,NaN,2175732.0,NaN,NaN,6002.0,5243.0,NaN,9736.0,245814.0,17332.0,NaN,Mike Reilly,mike reilly,QB,STL,1985-01-25,36.3,0.0,NaN,NaN,NaN,NaN,76.0,240.0,Central Washington,2025,Mike Reilly,Mike Reilly_9736,NaN
12036,9796,NaN,NaN,00-0026792,NaN,NaN,NaN,13027.0,NaN,NaN,1675238.0,WillPa01,NaN,6401.0,5595.0,NaN,9721.0,267416.0,NaN,NaN,Patrick Williams,patrick williams,WR,BAL,1986-01-13,35.3,0.0,NaN,NaN,NaN,NaN,73.0,204.0,Colorado,2025,Patrick Williams,Patrick Williams_9721,NaN
12038,9800,NaN,NaN,00-0024825,NaN,NaN,NaN,11142.0,NaN,NaN,1226291.0,NordJa00,NaN,5467.0,4608.0,NaN,8692.0,219960.0,NaN,NaN,Jake Nordin,jake nordin,TE,DET,1984-07-08,36.8,0.0,NaN,NaN,NaN,NaN,75.0,255.0,Northern Illinois,2025,Jake Nordin,Jake Nordin_8692,NaN
12039,9806,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6202.0,5498.0,NaN,9122.0,463275.0,NaN,NaN,Sam Giguere,sam giguere,WR,FA,1985-07-11,35.8,0.0,NaN,NaN,NaN,NaN,71.0,215.0,Sherbrooke,2025,Sam Giguere,Sam Giguere_9122,NaN


In [31]:
print(master_mismatch['mapped_ids'].isna().sum())

1409


In [32]:
master_mismatch.shape

(1409, 38)

In [33]:
master_mismatch.loc[(master_mismatch['mapped_ids'].isna()), 'mapped_ids'] = master_mismatch.loc[(master_mismatch['mapped_ids'].isna()), 'stats_id']
master_mismatch[master_mismatch['name'] == 'Ollie Gordon']

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids


In [34]:
master_mismatch.head()

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids
44,17074,4a975d4c-d7cd-45ef-b258-db45187140eb,26034.0,00-0040718,160766.0,12530.0,58204.0,4685415.0,NaN,NaN,28909849.0,HuntTr00,NaN,18578.0,NaN,1774.0,41787.0,0.0,26056.0,NaN,Travis Hunter,travis hunter,WR,JAC,2003-05-18,22.4,2025.0,1.0,2.0,2.0,NaN,73.0,185.0,Colorado,2025,Travis Hunter,Travis Hunter_41787,41787.0
190,17220,80dce1ba-c997-47e6-a37e-e92fefcbdc1b,NaN,00-0040200,143934.0,12713.0,58387.0,4569923.0,NaN,NaN,26699665.0,BorrAn00,NaN,18743.0,NaN,NaN,41980.0,0.0,26106.0,NaN,Andres Borregales,andres borregales,PK,NEP,2002-01-02,23.8,2025.0,6.0,6.0,182.0,NaN,71.0,202.0,Miami (FL),2025,Andres Borregales,Andres Borregales_41980,41980.0
358,16590,ec7b8cf0-0bb9-11ef-bb45-99966e93bb71,23002.0,NaN,NaN,12259.0,NaN,NaN,NaN,NaN,3123935.0,NaN,NaN,16999.0,NaN,NaN,41559.0,0.0,NaN,NaN,Taulia Tagovailoa,taulia tagovailoa,QB,FA,2000-02-15,25.7,2024.0,NaN,NaN,NaN,NaN,71.0,200.0,Maryland,2025,Taulia Tagovailoa,Taulia Tagovailoa_41559,41559.0
585,16818,8e6fc170-07d2-11ef-95d0-7db2ea91136a,NaN,NaN,NaN,12188.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41477.0,0.0,NaN,NaN,Justin Strong,justin strong,RB,FA,NaN,NaN,2024.0,NaN,NaN,NaN,NaN,72.0,205.0,Southern Illinois,2025,Justin Strong,Justin Strong_41477,41477.0
600,16833,fffee080-04ff-11ef-8a9a-5f3b29e3b4c8,26405.0,00-0039541,124169.0,11886.0,57660.0,4429153.0,41163.0,NaN,3171022.0,GrifLi00,NaN,17973.0,NaN,NaN,41163.0,0.0,NaN,1231735.0,Lideatrick Griffin,lideatrick griffin,WR,FA,2001-01-23,24.7,2024.0,NaN,NaN,NaN,NaN,70.0,181.0,Mississippi State,2025,Lideatrick Griffin,Lideatrick Griffin_41163,41163.0


In [35]:
updated_yahoo_maps = master_mismatch.set_index('match_key')['stats_id'].to_dict()
display(updated_yahoo_maps)

{'Travis Hunter_41787': 41787.0,
 'Andres Borregales_41980': 41980.0,
 'Taulia Tagovailoa_41559': 41559.0,
 'Justin Strong_41477': 41477.0,
 'Lideatrick Griffin_41163': 41163.0,
 'CJ Johnson_40461': 40461.0,
 'Scott Matlock_40225': 40225.0,
 'Demario Douglas_40222': 40222.0,
 'Grant Dubose_40251': 40251.0,
 'Keytaon Thompson_40592': 40592.0,
 'Mitchell Tinsley_40313': 40313.0,
 'EJ Perry_34346': 34346.0,
 'Dvonte Price_34521': 34521.0,
 'TJ Pledger_34691': 34691.0,
 'John Metchie_34063': 34063.0,
 'Chigoziem Okonkwo_34099': 34099.0,
 'Andrew Ogletree_34148': 34148.0,
 'Lujuan Winningham_34255': 34255.0,
 'John Parker Romo_34657': 34657.0,
 'Mike Strachan_33620': 33620.0,
 'BJ Emmons_33855': 33855.0,
 'AJ Dillon_32732': 32732.0,
 'Lamical Perine_32790': 32790.0,
 'Gabriel Davis_32798': 32798.0,
 'K.J. Hill_32890': 32890.0,
 'Jacob Breeland_32994': 32994.0,
 'LJ Scott_32569': 32569.0,
 'Marquise Brown_31857': 31857.0,
 'JJ Arcega-Whiteside_31889': 31889.0,
 'Tyron Johnson_32511': 32511.0

In [36]:
master_raw[master_raw['mapped_ids'].isna()].head()

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids
35,17065,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,Henry Parrish,henry parrish,RB,FA,NaN,NaN,2025.0,NaN,NaN,NaN,NaN,70.0,185.0,Mississippi,2025,Henry Parrish,Henry Parrish_<NA>,NaN
37,17067,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,John Emery,john emery,RB,FA,NaN,NaN,2025.0,NaN,NaN,NaN,NaN,72.0,220.0,LSU,2025,John Emery,John Emery_<NA>,NaN
44,17074,4a975d4c-d7cd-45ef-b258-db45187140eb,26034.0,00-0040718,160766.0,12530.0,58204.0,4685415.0,NaN,NaN,28909849.0,HuntTr00,NaN,18578.0,NaN,1774.0,41787.0,0.0,26056.0,NaN,Travis Hunter,travis hunter,WR,JAC,2003-05-18,22.4,2025.0,1.0,2.0,2.0,NaN,73.0,185.0,Colorado,2025,Travis Hunter,Travis Hunter_41787,NaN
59,17089,483872b0-2630-11f0-b7f1-df7c830143e7,27218.0,NaN,NaN,12460.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18559.0,NaN,NaN,NaN,0.0,26081.0,NaN,Antwane Wells,antwane wells,WR,NYG,2001-04-02,24.5,2025.0,NaN,NaN,NaN,NaN,72.0,205.0,Ole Miss,2025,Antwane Wells,Antwane Wells_<NA>,NaN
64,17094,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18456.0,NaN,NaN,NaN,0.0,NaN,NaN,Kyren Lacy,kyren lacy,XX,FA,NaN,NaN,2025.0,NaN,NaN,NaN,NaN,74.0,213.0,LSU,2025,Kyren Lacy,Kyren Lacy_<NA>,NaN


In [37]:
master_raw = master_raw[((master_raw['stats_id'] < 101000) & (master_raw['stats_id'] > 3720)) | (master_raw['stats_id'].isna())]

In [38]:
master_raw[master_raw['mapped_ids'].isna()].head()

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids
35,17065,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,Henry Parrish,henry parrish,RB,FA,NaN,NaN,2025.0,NaN,NaN,NaN,NaN,70.0,185.0,Mississippi,2025,Henry Parrish,Henry Parrish_<NA>,NaN
37,17067,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,John Emery,john emery,RB,FA,NaN,NaN,2025.0,NaN,NaN,NaN,NaN,72.0,220.0,LSU,2025,John Emery,John Emery_<NA>,NaN
44,17074,4a975d4c-d7cd-45ef-b258-db45187140eb,26034.0,00-0040718,160766.0,12530.0,58204.0,4685415.0,NaN,NaN,28909849.0,HuntTr00,NaN,18578.0,NaN,1774.0,41787.0,0.0,26056.0,NaN,Travis Hunter,travis hunter,WR,JAC,2003-05-18,22.4,2025.0,1.0,2.0,2.0,NaN,73.0,185.0,Colorado,2025,Travis Hunter,Travis Hunter_41787,NaN
59,17089,483872b0-2630-11f0-b7f1-df7c830143e7,27218.0,NaN,NaN,12460.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18559.0,NaN,NaN,NaN,0.0,26081.0,NaN,Antwane Wells,antwane wells,WR,NYG,2001-04-02,24.5,2025.0,NaN,NaN,NaN,NaN,72.0,205.0,Ole Miss,2025,Antwane Wells,Antwane Wells_<NA>,NaN
64,17094,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18456.0,NaN,NaN,NaN,0.0,NaN,NaN,Kyren Lacy,kyren lacy,XX,FA,NaN,NaN,2025.0,NaN,NaN,NaN,NaN,74.0,213.0,LSU,2025,Kyren Lacy,Kyren Lacy_<NA>,NaN


In [39]:
master_raw[master_raw['name'] == 'Antwane Wells']

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids
59,17089,483872b0-2630-11f0-b7f1-df7c830143e7,27218.0,NaN,NaN,12460.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18559.0,NaN,NaN,NaN,0.0,26081.0,NaN,Antwane Wells,antwane wells,WR,NYG,2001-04-02,24.5,2025.0,NaN,NaN,NaN,NaN,72.0,205.0,Ole Miss,2025,Antwane Wells,Antwane Wells_<NA>,NaN


In [40]:
master_raw.loc[(master_raw['match_key'].isin(updated_yahoo_maps.keys())) & (master_raw['mapped_ids'].isna()), 'mapped_ids'] = master_raw.loc[(master_raw['match_key'].isin(updated_yahoo_maps.keys())) & (master_raw['mapped_ids'].isna()), 'match_key'].map(updated_yahoo_maps)

In [41]:
master_raw[master_raw['mapped_ids'].isna()].head()

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids
35,17065,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,Henry Parrish,henry parrish,RB,FA,NaN,NaN,2025.0,NaN,NaN,NaN,NaN,70.0,185.0,Mississippi,2025,Henry Parrish,Henry Parrish_<NA>,NaN
37,17067,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,John Emery,john emery,RB,FA,NaN,NaN,2025.0,NaN,NaN,NaN,NaN,72.0,220.0,LSU,2025,John Emery,John Emery_<NA>,NaN
59,17089,483872b0-2630-11f0-b7f1-df7c830143e7,27218.0,NaN,NaN,12460.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18559.0,NaN,NaN,NaN,0.0,26081.0,NaN,Antwane Wells,antwane wells,WR,NYG,2001-04-02,24.5,2025.0,NaN,NaN,NaN,NaN,72.0,205.0,Ole Miss,2025,Antwane Wells,Antwane Wells_<NA>,NaN
64,17094,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18456.0,NaN,NaN,NaN,0.0,NaN,NaN,Kyren Lacy,kyren lacy,XX,FA,NaN,NaN,2025.0,NaN,NaN,NaN,NaN,74.0,213.0,LSU,2025,Kyren Lacy,Kyren Lacy_<NA>,NaN
80,17110,NaN,NaN,NaN,NaN,11594.0,NaN,NaN,NaN,NaN,2965087.0,NaN,NaN,18541.0,NaN,NaN,NaN,0.0,NaN,NaN,Brant Kuithe,brant kuithe,TE,FA,NaN,NaN,2025.0,NaN,NaN,NaN,NaN,74.0,236.0,Utah,2025,Brant Kuithe,Brant Kuithe_<NA>,NaN


In [42]:
master_raw.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5432 entries, 0 to 12143
Data columns (total 38 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   mfl_id            5432 non-null   int64  
 1   sportradar_id     3263 non-null   object 
 2   fantasypros_id    2489 non-null   float64
 3   gsis_id           3339 non-null   object 
 4   pff_id            3146 non-null   float64
 5   sleeper_id        2844 non-null   float64
 6   nfl_id            3091 non-null   float64
 7   espn_id           3457 non-null   float64
 8   yahoo_id          2684 non-null   float64
 9   fleaflicker_id    493 non-null    float64
 10  cbs_id            4130 non-null   float64
 11  pfr_id            3917 non-null   object 
 12  cfbref_id         2171 non-null   object 
 13  rotowire_id       4749 non-null   float64
 14  rotoworld_id      3257 non-null   float64
 15  ktc_id            394 non-null    float64
 16  stats_id          4339 non-null   float64
 17 

In [43]:
master_raw[~master_raw['cleaned_name'].isin(yahoo_players['cleaned_name'])]

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids
35,17065,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,Henry Parrish,henry parrish,RB,FA,NaN,NaN,2025.0,NaN,NaN,NaN,NaN,70.0,185.0,Mississippi,2025,Henry Parrish,Henry Parrish_<NA>,NaN
37,17067,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,John Emery,john emery,RB,FA,NaN,NaN,2025.0,NaN,NaN,NaN,NaN,72.0,220.0,LSU,2025,John Emery,John Emery_<NA>,NaN
64,17094,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18456.0,NaN,NaN,NaN,0.0,NaN,NaN,Kyren Lacy,kyren lacy,XX,FA,NaN,NaN,2025.0,NaN,NaN,NaN,NaN,74.0,213.0,LSU,2025,Kyren Lacy,Kyren Lacy_<NA>,NaN
80,17110,NaN,NaN,NaN,NaN,11594.0,NaN,NaN,NaN,NaN,2965087.0,NaN,NaN,18541.0,NaN,NaN,NaN,0.0,NaN,NaN,Brant Kuithe,brant kuithe,TE,FA,NaN,NaN,2025.0,NaN,NaN,NaN,NaN,74.0,236.0,Utah,2025,Brant Kuithe,Brant Kuithe_<NA>,NaN
190,17220,80dce1ba-c997-47e6-a37e-e92fefcbdc1b,NaN,00-0040200,143934.0,12713.0,58387.0,4569923.0,NaN,NaN,26699665.0,BorrAn00,NaN,18743.0,NaN,NaN,41980.0,0.0,26106.0,NaN,Andres Borregales,andres borregales,PK,NEP,2002-01-02,23.8,2025.0,6.0,6.0,182.0,NaN,71.0,202.0,Miami (FL),2025,Andres Borregales,Andres Borregales_41980,41980.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12139,968,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sean Brewer,sean brewer,TE,CIN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025,Sean Brewer,Sean Brewer_<NA>,NaN
12140,972,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Doug Brien,doug brien,PK,NOS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025,Doug Brien,Doug Brien_<NA>,NaN
12141,978,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jeremy Brigham,jeremy brigham,TE,OAK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025,Jeremy Brigham,Jeremy Brigham_<NA>,NaN
12142,984,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vincent Brisby,vincent brisby,WR,FA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025,Vincent Brisby,Vincent Brisby_<NA>,NaN


In [43]:
master_raw[master_raw['name'] == 'Travis Hunter']

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,cleaned_name,match_key,mapped_ids
44,17074,4a975d4c-d7cd-45ef-b258-db45187140eb,26034.0,00-0040718,160766.0,12530.0,58204.0,4685415.0,NaN,NaN,28909849.0,HuntTr00,NaN,18578.0,NaN,1774.0,41787.0,0.0,26056.0,NaN,Travis Hunter,travis hunter,WR,JAC,2003-05-18,22.4,2025.0,1.0,2.0,2.0,NaN,73.0,185.0,Colorado,2025,Travis Hunter,Travis Hunter_41787,41787.0


In [82]:
master_raw['mapped_ids'].isna().sum()

np.int64(1192)

In [84]:
((master_raw['mapped_ids'].isna()) & (master_raw['stats_id'].isna())).sum()

np.int64(1093)

In [85]:
master_cln_raw = master_raw.copy()


In [86]:
master_cln_raw.loc[(master_cln_raw['mapped_ids'].isna()) & (master_cln_raw['stats_id'].notna()), 'mapped_ids'] = master_cln_raw.loc[(master_cln_raw['mapped_ids'].isna()) & (master_cln_raw['stats_id'].notna()), 'stats_id']


In [87]:
master_cln_raw.head()

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,match_key,mapped_ids
0,17030,3c76cab3-3df2-43dd-acaa-57e055bd32d0,24755.0,00-0040676,133244.0,12522.0,58203.0,4688380.0,NaN,NaN,3168422.0,WardCa00,NaN,16997.0,NaN,1730.0,41786.0,0.0,25323.0,NaN,Cam Ward,cam ward,QB,TEN,2002-05-25,23.4,2025.0,1.0,1.0,1.0,NaN,74.0,219.0,Miami (FL),2025,Cam Ward_41786,41786.0
1,17031,270e09bc-8bf8-44b0-87ed-1fd014de4ab7,25968.0,00-0040668,131396.0,12524.0,58349.0,4432762.0,NaN,NaN,26710004.0,SandSh00,NaN,18479.0,NaN,1731.0,41930.0,0.0,26079.0,NaN,Shedeur Sanders,shedeur sanders,QB,CLE,2002-02-07,23.7,2025.0,5.0,6.0,144.0,NaN,74.0,212.0,Colorado,2025,Shedeur Sanders_41930,41930.0
2,17032,53848cba-bcad-4cd0-bb35-0a1f6ed111d1,23160.0,00-0040691,146409.0,12508.0,58227.0,4689114.0,NaN,NaN,26694019.0,DartJa00,NaN,18574.0,NaN,1732.0,41810.0,0.0,26082.0,NaN,Jaxson Dart,jaxson dart,QB,NYG,2003-05-13,22.4,2025.0,1.0,25.0,25.0,NaN,74.0,223.0,Ole Miss,2025,Jaxson Dart_41810,41810.0
3,17033,b0eb5b67-4ef2-462d-bfe7-e73bcce1f6b0,22884.0,00-0040673,146590.0,12510.0,58297.0,4432734.0,NaN,NaN,26694943.0,MilrJa00,NaN,18477.0,NaN,1733.0,41882.0,0.0,25882.0,NaN,Jalen Milroe,jalen milroe,QB,SEA,2002-12-13,22.8,2025.0,3.0,28.0,92.0,NaN,74.0,216.0,Alabama,2025,Jalen Milroe_41882,41882.0
4,17034,d89f0384-21a6-44e4-8aa6-324cb2103bf6,23680.0,00-0040234,127253.0,12500.0,58436.0,4889929.0,NaN,NaN,26746721.0,EwerQu00,NaN,18505.0,NaN,1734.0,42037.0,0.0,26048.0,NaN,Quinn Ewers,quinn ewers,QB,MIA,2003-05-13,22.4,2025.0,7.0,15.0,231.0,NaN,74.0,209.0,Texas,2025,Quinn Ewers_42037,42037.0


In [88]:
master_cln_raw[master_cln_raw['name'] == 'Ollie Gordon']

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,match_key,mapped_ids
29,17059,42753bb4-a416-4d40-bf81-f20d2181be2f,25984.0,00-0040198,157792.0,12495.0,58384.0,4711533.0,NaN,NaN,28873887.0,GordOl00,NaN,18450.0,NaN,1759.0,41969.0,0.0,25953.0,NaN,Ollie Gordon,ollie gordon,RB,MIA,2004-01-15,21.8,2025.0,6.0,3.0,179.0,NaN,74.0,225.0,Oklahoma State,2025,Ollie Gordon_41969,41969.0


In [94]:
master_cln_raw[(master_cln_raw['mapped_ids'].isna()) & (master_cln_raw['match_key'].str.contains('<NA>'))]

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,match_key,mapped_ids
35,17065,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,Henry Parrish,henry parrish,RB,FA,NaN,NaN,2025.0,NaN,NaN,NaN,NaN,70.0,185.0,Mississippi,2025,Henry Parrish_<NA>,NaN
37,17067,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,John Emery,john emery,RB,FA,NaN,NaN,2025.0,NaN,NaN,NaN,NaN,72.0,220.0,LSU,2025,John Emery_<NA>,NaN
59,17089,483872b0-2630-11f0-b7f1-df7c830143e7,27218.0,NaN,NaN,12460.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18559.0,NaN,NaN,NaN,0.0,26081.0,NaN,Antwane Wells,antwane wells,WR,NYG,2001-04-02,24.5,2025.0,NaN,NaN,NaN,NaN,72.0,205.0,Ole Miss,2025,Antwane Wells_<NA>,NaN
64,17094,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18456.0,NaN,NaN,NaN,0.0,NaN,NaN,Kyren Lacy,kyren lacy,XX,FA,NaN,NaN,2025.0,NaN,NaN,NaN,NaN,74.0,213.0,LSU,2025,Kyren Lacy_<NA>,NaN
80,17110,NaN,NaN,NaN,NaN,11594.0,NaN,NaN,NaN,NaN,2965087.0,NaN,NaN,18541.0,NaN,NaN,NaN,0.0,NaN,NaN,Brant Kuithe,brant kuithe,TE,FA,NaN,NaN,2025.0,NaN,NaN,NaN,NaN,74.0,236.0,Utah,2025,Brant Kuithe_<NA>,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12139,968,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sean Brewer,sean brewer,TE,CIN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025,Sean Brewer_<NA>,NaN
12140,972,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Doug Brien,doug brien,PK,NOS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025,Doug Brien_<NA>,NaN
12141,978,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jeremy Brigham,jeremy brigham,TE,OAK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025,Jeremy Brigham_<NA>,NaN
12142,984,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vincent Brisby,vincent brisby,WR,FA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025,Vincent Brisby_<NA>,NaN


In [ ]:
# master_cln_raw.loc[master_cln_raw['mapped_ids'].isna(), 'mapped_ids'] = master_cln_raw.loc[master_cln_raw['mapped_ids'].isna(), 'name'].map(yahoo_id_maps)

In [93]:
master_cln_raw[(master_cln_raw['mapped_ids'].isna())]

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,match_key,mapped_ids
35,17065,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,Henry Parrish,henry parrish,RB,FA,NaN,NaN,2025.0,NaN,NaN,NaN,NaN,70.0,185.0,Mississippi,2025,Henry Parrish_<NA>,NaN
37,17067,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,John Emery,john emery,RB,FA,NaN,NaN,2025.0,NaN,NaN,NaN,NaN,72.0,220.0,LSU,2025,John Emery_<NA>,NaN
59,17089,483872b0-2630-11f0-b7f1-df7c830143e7,27218.0,NaN,NaN,12460.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18559.0,NaN,NaN,NaN,0.0,26081.0,NaN,Antwane Wells,antwane wells,WR,NYG,2001-04-02,24.5,2025.0,NaN,NaN,NaN,NaN,72.0,205.0,Ole Miss,2025,Antwane Wells_<NA>,NaN
64,17094,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18456.0,NaN,NaN,NaN,0.0,NaN,NaN,Kyren Lacy,kyren lacy,XX,FA,NaN,NaN,2025.0,NaN,NaN,NaN,NaN,74.0,213.0,LSU,2025,Kyren Lacy_<NA>,NaN
80,17110,NaN,NaN,NaN,NaN,11594.0,NaN,NaN,NaN,NaN,2965087.0,NaN,NaN,18541.0,NaN,NaN,NaN,0.0,NaN,NaN,Brant Kuithe,brant kuithe,TE,FA,NaN,NaN,2025.0,NaN,NaN,NaN,NaN,74.0,236.0,Utah,2025,Brant Kuithe_<NA>,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12139,968,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sean Brewer,sean brewer,TE,CIN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025,Sean Brewer_<NA>,NaN
12140,972,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Doug Brien,doug brien,PK,NOS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025,Doug Brien_<NA>,NaN
12141,978,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jeremy Brigham,jeremy brigham,TE,OAK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025,Jeremy Brigham_<NA>,NaN
12142,984,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vincent Brisby,vincent brisby,WR,FA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025,Vincent Brisby_<NA>,NaN


In [95]:
# Create name cleaning funciton
def clean_name(name):
    if pd.isna(name) or name is None:
        return name
    return re.sub(r'\s+(Jr\.?|Sr\.?|III|II|IV|V)(\s|$|,)', '', str(name), flags=re.IGNORECASE).strip()

In [96]:
# create function to cehck if dupes will be created and list where
def check_cleaning_duplicates(original_dict, clean_function):
    """Check if cleaning will create duplicate keys"""
    
    # Map: cleaned_name -> list of original names
    cleaned_to_original = defaultdict(list)
    
    for original_name in original_dict.keys():
        cleaned = clean_function(original_name)
        cleaned_to_original[cleaned].append(original_name)
    
    # Find cleaned names that have multiple originals
    duplicates = {k: v for k, v in cleaned_to_original.items() if len(v) > 1}
    
    if duplicates:
        print(f"Found {len(duplicates)} names that will become duplicates after cleaning:\n")
        for cleaned_name, original_names in duplicates.items():
            print(f'{cleaned_name}')
            for orig in original_names:
                print(f"    - '{orig}' (ID: {original_dict[orig]})")
            print()
        return duplicates
    else:
        print("No duplicates will be created after cleaning")
        return {}

In [98]:
yahoo_id_maps

{'Adam Vinatieri_3727': 3727,
 'Tony Gonzalez_3950': 3950,
 'Peyton Manning_4256': 4256,
 'Phil Dawson_4269': 4269,
 'Matt Hasselbeck_4416': 4416,
 'David Akers_4587': 4587,
 'Sebastian Janikowski_5046': 5046,
 'Tom Brady_5228': 5228,
 'Mike Leach_5381': 5381,
 'Rian Lindell_5388': 5388,
 'Jay Feely_5447': 5447,
 'Michael Vick_5448': 5448,
 'Santana Moss_5463': 5463,
 'Reggie Wayne_5477': 5477,
 'Drew Brees_5479': 5479,
 'Steve Smith_5521': 5521,
 'Shayne Graham_5854': 5854,
 'Josh McCown_5967': 5967,
 'David Garrard_5994': 5994,
 'Billy Cundiff_6154': 6154,
 'Shaun Hill_6169': 6169,
 'Matt Bryant_6243': 6243,
 'Carson Palmer_6337': 6337,
 'Andre Johnson_6339': 6339,
 'Rex Grossman_6358': 6358,
 'Willis McGahee_6359': 6359,
 'Dallas Clark_6360': 6360,
 'Anquan Boldin_6390': 6390,
 'Jason Witten_6405': 6405,
 'Nate Burleson_6407': 6407,
 'Brandon Lloyd_6460': 6460,
 'Josh Brown_6558': 6558,
 'Tony Romo_6624': 6624,
 'Antonio Gates_6663': 6663,
 'Kassim Osgood_6664': 6664,
 'Eli Manning_

In [97]:
# Run function

check_cleaning_duplicates(yahoo_id_maps, clean_name)

No duplicates will be created after cleaning


{}

In [ ]:
# Create mapping with cleaned names
cln_yahoo_id_maps = {clean_name(key): value for key, value in yahoo_id_maps.items()}

In [ ]:

# Add ID maps woth standardized names
master_cln_raw.loc[master_cln_raw['mapped_ids'].isna(), 'mapped_ids'] = master_cln_raw.loc[master_cln_raw['mapped_ids'].isna(), 'name'].map(cln_yahoo_id_maps)

In [37]:
cln_yahoo_id_maps['Rob Bironas']

7434

In [38]:
master_cln_raw[master_cln_raw['name'] == 'Rob Bironas']

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,mapped_ids


In [ ]:
master_cln_raw[master_cln_raw['mapped_ids'].isna()]

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,mapped_ids
35,17065,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,Henry Parrish,henry parrish,RB,FA,NaN,NaN,2025.0,NaN,NaN,NaN,NaN,70.0,185.0,Mississippi,2025,NaN
37,17067,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,John Emery,john emery,RB,FA,NaN,NaN,2025.0,NaN,NaN,NaN,NaN,72.0,220.0,LSU,2025,NaN
80,17110,NaN,NaN,NaN,NaN,11594.0,NaN,NaN,NaN,NaN,2965087.0,NaN,NaN,18541.0,NaN,NaN,NaN,0.0,NaN,NaN,Brant Kuithe,brant kuithe,TE,FA,NaN,NaN,2025.0,NaN,NaN,NaN,NaN,74.0,236.0,Utah,2025,NaN
370,16602,NaN,NaN,NaN,NaN,11578.0,NaN,NaN,NaN,NaN,3169578.0,NaN,NaN,17857.0,NaN,NaN,NaN,0.0,NaN,NaN,Miyan Williams,miyan williams,RB,FA,NaN,NaN,2024.0,NaN,NaN,NaN,NaN,69.0,226.0,Ohio State,2025,NaN
1084,15700,NaN,NaN,NaN,NaN,8216.0,NaN,NaN,NaN,NaN,2966363.0,NaN,NaN,15970.0,NaN,NaN,NaN,0.0,NaN,NaN,Kaleb Eleby,kaleb eleby,QB,FA,2000-03-13,23.0,2022.0,NaN,NaN,NaN,NaN,73.0,208.0,Western Michigan,2025,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12139,968,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sean Brewer,sean brewer,TE,CIN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025,NaN
12140,972,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Doug Brien,doug brien,PK,NOS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025,NaN
12141,978,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jeremy Brigham,jeremy brigham,TE,OAK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025,NaN
12142,984,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vincent Brisby,vincent brisby,WR,FA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025,NaN


: 

: 

In [ ]:
master_cln_raw[master_cln_raw ['name'] == 'Travis Hunter']

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,mapped_ids
44,17074,4a975d4c-d7cd-45ef-b258-db45187140eb,26034.0,00-0040718,160766.0,12530.0,58204.0,4685415.0,NaN,NaN,28909849.0,HuntTr00,NaN,18578.0,NaN,1774.0,41787.0,0.0,26056.0,NaN,Travis Hunter,travis hunter,WR,JAC,2003-05-18,22.4,2025.0,1.0,2.0,2.0,NaN,73.0,185.0,Colorado,2025,99001.0


: 

: 

In [ ]:
# Any missing yahoo ID values are now because those players were not in the yahoo ID Set, can drop these rows
master_cln_raw = master_cln_raw.dropna(subset=['mapped_ids'])
master_cln_raw['mapped_ids'] = master_cln_raw['mapped_ids'].astype(int)
master_cln_raw.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4472 entries, 0 to 12138
Data columns (total 36 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   mfl_id            4472 non-null   int64  
 1   sportradar_id     3261 non-null   object 
 2   fantasypros_id    2487 non-null   float64
 3   gsis_id           3345 non-null   object 
 4   pff_id            3142 non-null   float64
 5   sleeper_id        2837 non-null   float64
 6   nfl_id            3089 non-null   float64
 7   espn_id           3464 non-null   float64
 8   yahoo_id          2679 non-null   float64
 9   fleaflicker_id    495 non-null    float64
 10  cbs_id            4150 non-null   float64
 11  pfr_id            3608 non-null   object 
 12  cfbref_id         1906 non-null   object 
 13  rotowire_id       4326 non-null   float64
 14  rotoworld_id      3275 non-null   float64
 15  ktc_id            394 non-null    float64
 16  stats_id          4438 non-null   float64
 17 

: 

: 

In [ ]:
master_cln_raw.head()

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,mapped_ids
0,17030,3c76cab3-3df2-43dd-acaa-57e055bd32d0,24755.0,00-0040676,133244.0,12522.0,58203.0,4688380.0,NaN,NaN,3168422.0,WardCa00,NaN,16997.0,NaN,1730.0,41786.0,0.0,25323.0,NaN,Cam Ward,cam ward,QB,TEN,2002-05-25,23.4,2025.0,1.0,1.0,1.0,NaN,74.0,219.0,Miami (FL),2025,41786
1,17031,270e09bc-8bf8-44b0-87ed-1fd014de4ab7,25968.0,00-0040668,131396.0,12524.0,58349.0,4432762.0,NaN,NaN,26710004.0,SandSh00,NaN,18479.0,NaN,1731.0,41930.0,0.0,26079.0,NaN,Shedeur Sanders,shedeur sanders,QB,CLE,2002-02-07,23.7,2025.0,5.0,6.0,144.0,NaN,74.0,212.0,Colorado,2025,41930
2,17032,53848cba-bcad-4cd0-bb35-0a1f6ed111d1,23160.0,00-0040691,146409.0,12508.0,58227.0,4689114.0,NaN,NaN,26694019.0,DartJa00,NaN,18574.0,NaN,1732.0,41810.0,0.0,26082.0,NaN,Jaxson Dart,jaxson dart,QB,NYG,2003-05-13,22.4,2025.0,1.0,25.0,25.0,NaN,74.0,223.0,Ole Miss,2025,41810
3,17033,b0eb5b67-4ef2-462d-bfe7-e73bcce1f6b0,22884.0,00-0040673,146590.0,12510.0,58297.0,4432734.0,NaN,NaN,26694943.0,MilrJa00,NaN,18477.0,NaN,1733.0,41882.0,0.0,25882.0,NaN,Jalen Milroe,jalen milroe,QB,SEA,2002-12-13,22.8,2025.0,3.0,28.0,92.0,NaN,74.0,216.0,Alabama,2025,41882
4,17034,d89f0384-21a6-44e4-8aa6-324cb2103bf6,23680.0,00-0040234,127253.0,12500.0,58436.0,4889929.0,NaN,NaN,26746721.0,EwerQu00,NaN,18505.0,NaN,1734.0,42037.0,0.0,26048.0,NaN,Quinn Ewers,quinn ewers,QB,MIA,2003-05-13,22.4,2025.0,7.0,15.0,231.0,NaN,74.0,209.0,Texas,2025,42037


: 

: 

In [ ]:
master_cln_raw.loc[master_cln_raw['stats_id'].isna(), 'stats_id'] = master_cln_raw.loc[master_cln_raw['stats_id'].isna(), 'mapped_ids']

: 

: 

In [ ]:
master_cln_raw['stats_id'].isna().sum()

np.int64(0)

: 

: 

In [ ]:
master_cln_raw.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4472 entries, 0 to 12138
Data columns (total 36 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   mfl_id            4472 non-null   int64  
 1   sportradar_id     3261 non-null   object 
 2   fantasypros_id    2487 non-null   float64
 3   gsis_id           3345 non-null   object 
 4   pff_id            3142 non-null   float64
 5   sleeper_id        2837 non-null   float64
 6   nfl_id            3089 non-null   float64
 7   espn_id           3464 non-null   float64
 8   yahoo_id          2679 non-null   float64
 9   fleaflicker_id    495 non-null    float64
 10  cbs_id            4150 non-null   float64
 11  pfr_id            3608 non-null   object 
 12  cfbref_id         1906 non-null   object 
 13  rotowire_id       4326 non-null   float64
 14  rotoworld_id      3275 non-null   float64
 15  ktc_id            394 non-null    float64
 16  stats_id          4472 non-null   float64
 17 

: 

: 

In [ ]:
master_cln_raw['stats_id'].nunique()

4442

: 

: 

In [ ]:
master_cln_raw[master_cln_raw['stats_id'].duplicated(keep=False)].sort_values(by='stats_id').head(60)

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,mapped_ids
9231,3291,aae6d92e-5f28-43ee-b0dc-522e80e99f76,9507.0,00-0010346,327.0,294.0,2501863.0,1428.0,4256.0,NaN,12531.0,MannPe00,peyton-manning-1,902.0,1493.0,NaN,4256.0,24375.0,7328.0,NaN,Peyton Manning,peyton manning,QB,FA,1976-03-24,45.1,1998.0,1.0,1.0,1.0,NaN,77.0,230.0,Tennessee,2025,4256
10556,5274,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4256.0,0.0,NaN,NaN,Peyton Manning,peyton manning,QB,IND,NaN,NaN,1987.0,NaN,NaN,NaN,NaN,73.0,218.0,Pittsburgh,2025,4256
9844,3852,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4416.0,0.0,NaN,NaN,Matt Hasselbeck,matt hasselbeck,QB,SEA,NaN,NaN,1995.0,6.0,5.0,NaN,NaN,NaN,NaN,California,2025,4416
9358,3445,7d8eba61-208d-4d91-86cd-704ad05cb7f4,9439.0,00-0007091,408.0,36.0,2501078.0,1575.0,4416.0,NaN,12429.0,HassMa00,matt-hasselbeck-1,949.0,519.0,NaN,4416.0,24535.0,1034.0,NaN,Matt Hasselbeck,matt hasselbeck,QB,FA,1975-09-25,45.6,1998.0,6.0,34.0,187.0,Hasselbeck,76.0,235.0,Boston College,2025,4416
9384,3579,8259ba2b-d4ae-4290-964f-903fa09b1c4f,NaN,00-0000108,295.0,NaN,2499370.0,1716.0,NaN,NaN,1666.0,akersdav01,NaN,1052.0,721.0,NaN,4587.0,24706.0,NaN,NaN,David Akers,david akers,PK,FA,1974-12-09,46.4,1998.0,NaN,NaN,NaN,NaN,70.0,200.0,Louisville,2025,4587
12056,102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4587.0,NaN,NaN,NaN,David Akers,david akers,PK,PHI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025,4587
10191,4392,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5046.0,0.0,NaN,NaN,Sebastian Janikowski,sebastian janikowski,PK,OAK,NaN,NaN,1992.0,7.0,24.0,NaN,NaN,NaN,NaN,Houston,2025,5046
8799,5666,480277d1-47c9-44df-969e-038a84cd0fea,9560.0,00-0019646,595.0,118.0,2504257.0,2148.0,5046.0,NaN,187389.0,janikseb01,sebastian-janikowski-1,1299.0,819.0,NaN,5046.0,25165.0,3253.0,NaN,Sebastian Janikowski,sebastian janikowski,PK,FA,1978-03-02,43.2,2000.0,1.0,17.0,17.0,NaN,73.0,260.0,Florida State,2025,5046
8960,5848,41c44740-d0f6-44ab-8347-3b5d515e5ecf,9603.0,00-0019596,698.0,167.0,2504211.0,2330.0,5228.0,309.0,187741.0,BradTo00,tom-brady-1,1350.0,1163.0,NaN,5228.0,25347.0,4314.0,25347.0,Tom Brady,tom brady,QB,FA,1977-08-03,48.2,2000.0,6.0,33.0,199.0,NaN,76.0,225.0,Michigan,2025,5228
12135,948,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5228.0,NaN,NaN,NaN,Tom Brady,tom brady,QB,NEP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025,5228


: 

: 

In [ ]:
master_cln_raw = (master_cln_raw.sort_values(by=['stats_id', 'sportradar_id'], na_position='last').drop_duplicates(subset='stats_id', keep='first'))

: 

: 

In [ ]:
master_cln_raw.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4442 entries, 10536 to 46
Data columns (total 36 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   mfl_id            4442 non-null   int64  
 1   sportradar_id     3260 non-null   object 
 2   fantasypros_id    2486 non-null   float64
 3   gsis_id           3344 non-null   object 
 4   pff_id            3141 non-null   float64
 5   sleeper_id        2836 non-null   float64
 6   nfl_id            3088 non-null   float64
 7   espn_id           3463 non-null   float64
 8   yahoo_id          2678 non-null   float64
 9   fleaflicker_id    495 non-null    float64
 10  cbs_id            4149 non-null   float64
 11  pfr_id            3598 non-null   object 
 12  cfbref_id         1896 non-null   object 
 13  rotowire_id       4318 non-null   float64
 14  rotoworld_id      3274 non-null   float64
 15  ktc_id            393 non-null    float64
 16  stats_id          4442 non-null   float64
 17

: 

: 

In [ ]:
master_cln_raw['stats_id'].nunique()

4442

: 

: 

In [ ]:
master_cln_raw['mapped_ids'].nunique()

4410

: 

: 

In [ ]:
master_cln_raw.loc[master_cln_raw['mapped_ids'].duplicated(keep=False), 'mapped_ids'] = master_cln_raw.loc[master_cln_raw['mapped_ids'].duplicated(keep=False), 'stats_id']

: 

: 

In [ ]:
master_cln_raw['mapped_ids'].nunique()

4442

: 

: 

In [ ]:
master_cln_raw[master_cln_raw['name'] == 'Rob Bironas']

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,mapped_ids


: 

: 

In [ ]:
master_copy = master_cln_raw.copy()
full_copy = yahoo_players.merge(master_copy, how='left', left_on='player_id', right_on='mapped_ids')
full_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14096 entries, 0 to 14095
Data columns (total 61 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   bye_weeks.week            14096 non-null  int64  
 1   display_position          14096 non-null  object 
 2   editorial_player_key      14096 non-null  object 
 3   editorial_team_abbr       14096 non-null  object 
 4   editorial_team_full_name  14096 non-null  object 
 5   editorial_team_key        14096 non-null  object 
 6   editorial_team_url        14096 non-null  object 
 7   eligible_positions        14096 non-null  object 
 8   headshot.size             14096 non-null  object 
 9   headshot.url              14096 non-null  object 
 10  image_url                 14096 non-null  object 
 11  is_keeper.status          14096 non-null  bool   
 12  is_keeper.cost            14096 non-null  bool   
 13  is_keeper.kept            14096 non-null  bool   
 14  is_und

: 

: 

In [ ]:
full_copy[(full_copy['stats_id'].isna())]

,bye_weeks.week,display_position,editorial_player_key,editorial_team_abbr,editorial_team_full_name,editorial_team_key,editorial_team_url,eligible_positions,headshot.size,headshot.url,image_url,is_keeper.status,is_keeper.cost,is_keeper.kept,is_undroppable,name.ascii_first,name.first,name.full,name.last,player_id,player_key,position_type,primary_position,url,season,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,mapped_ids
52,10,RB,nfl.p.7178,Hou,Houston Texans,nfl.t.34,https://sports.yahoo.com/nfl/teams/houston/,['RB'],small,https://s.yimg.com/iu/api/res/1.2/QYw6.yBPo_XN...,https://s.yimg.com/iu/api/res/1.2/QYw6.yBPo_XN...,False,False,False,0,Ronnie,Ronnie,Ronnie Brown,Brown,7178,331.p.7178,O,RB,https://sports.yahoo.com/nfl/players/7178,2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69,9,K,nfl.p.7434,Ten,Tennessee Titans,nfl.t.10,https://sports.yahoo.com/nfl/teams/tennessee/,['K'],small,https://s.yimg.com/iu/api/res/1.2/etIu4LtcHSr7...,https://s.yimg.com/iu/api/res/1.2/etIu4LtcHSr7...,False,False,False,0,Rob,Rob,Rob Bironas,Bironas,7434,331.p.7434,K,K,https://sports.yahoo.com/nfl/players/7434,2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
122,10,WR,nfl.p.8327,LAC,Los Angeles Chargers,nfl.t.24,https://sports.yahoo.com/nfl/teams/la-chargers/,['WR'],small,https://s.yimg.com/iu/api/res/1.2/3guLflRZ8IWP...,https://s.yimg.com/iu/api/res/1.2/3guLflRZ8IWP...,False,False,False,0,Jacoby,Jacoby,Jacoby Jones,Jones,8327,331.p.8327,O,WR,https://sports.yahoo.com/nfl/players/8327,2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
348,12,WR,nfl.p.24746,Car,Carolina Panthers,nfl.t.29,https://sports.yahoo.com/nfl/teams/carolina/,['WR'],small,https://s.yimg.com/iu/api/res/1.2/M8gJ9e0k7poq...,https://s.yimg.com/iu/api/res/1.2/M8gJ9e0k7poq...,False,False,False,0,Teddy,Teddy,Teddy Williams,Williams,24746,331.p.24746,O,WR,https://sports.yahoo.com/nfl/players/24746,2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
363,4,RB,nfl.p.24825,Ari,Arizona Cardinals,nfl.t.22,https://sports.yahoo.com/nfl/teams/arizona/,['RB'],small,https://s.yimg.com/iu/api/res/1.2/B8vwtafB1pNJ...,https://s.yimg.com/iu/api/res/1.2/B8vwtafB1pNJ...,False,False,False,0,Ryan,Ryan,Ryan Williams,Williams,24825,331.p.24825,O,RB,https://sports.yahoo.com/nfl/players/24825,2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14091,12,DEF,nfl.p.100028,Was,Washington Commanders,nfl.t.28,https://sports.yahoo.com/nfl/teams/washington/,['DEF'],small,https://s.yimg.com/cv/apiv2/default/nfl/202009...,https://s.yimg.com/cv/apiv2/default/nfl/202009...,False,False,False,0,Washington,Washington,Washington Commanders,Commanders,100028,461.p.100028,DT,DEF,https://sports.yahoo.com/nfl/teams/washington/,2025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14092,14,DEF,nfl.p.100029,Car,Carolina Panthers,nfl.t.29,https://sports.yahoo.com/nfl/teams/carolina/,['DEF'],small,https://s.yimg.com/c

: 

: 

In [ ]:
# def position_checker():
#     master_copy = master_raw.copy()
#     master_copy.loc[(master_copy['yahoo_id'].isna()) & (master_copy['stats_id'].notna()), 'yahoo_id'] = master_copy.loc[(master_copy['yahoo_id'].isna()) & (master_copy['stats_id'].notna()), 'stats_id']
#     master_copy.loc[(master_copy['yahoo_id'] != master_copy['stats_id']), 'yahoo_id'] = master_copy.loc[(master_copy['yahoo_id'] != master_copy['stats_id']), 'stats_id']
#     master_copy.loc[(master_copy['yahoo_id'] != master_copy['stats_id']), 'yahoo_id'] = master_copy.loc[(master_copy['yahoo_id'] != master_copy['stats_id']), 'stats_id']
#     full_copy = yahoo_players.merge(master_copy, how='left', left_on='player_id', right_on='yahoo_id')
#     # full_copy.loc[(full_copy['primary_position'] == 'DEF'), 'yahoo_id'] = full_copy.loc[(full_copy['primary_position'] == 'DEF'), 'player_id']
#     # mismatches = full_copy[full_copy['primary_position'] != full_copy['position']]
#     return full_copy

: 

: 

In [ ]:
____________________

NameError: name '____________________' is not defined

: 

: 

In [ ]:
# result = position_checker()

: 

: 

In [ ]:
full_copy[full_copy['stats_id'].isna()]

,bye_weeks.week,display_position,editorial_player_key,editorial_team_abbr,editorial_team_full_name,editorial_team_key,editorial_team_url,eligible_positions,headshot.size,headshot.url,image_url,is_keeper.status,is_keeper.cost,is_keeper.kept,is_undroppable,name.ascii_first,name.first,name.full,name.last,player_id,player_key,position_type,primary_position,url,season,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season,mapped_ids
3,4,QB,nfl.p.4256,Den,Denver Broncos,nfl.t.7,https://sports.yahoo.com/nfl/teams/denver/,['QB'],small,https://s.yimg.com/iu/api/res/1.2/UCWtIm9kEbbH...,https://s.yimg.com/iu/api/res/1.2/UCWtIm9kEbbH...,False,False,False,0,Peyton,Peyton,Peyton Manning,Manning,4256,331.p.4256,O,QB,https://sports.yahoo.com/nfl/players/4256,2014,5274.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,Peyton Manning,peyton manning,QB,IND,NaN,NaN,1987.0,NaN,NaN,NaN,NaN,73.0,218.0,Pittsburgh,2025.0,4256.0
6,10,QB,nfl.p.4416,Ind,Indianapolis Colts,nfl.t.11,https://sports.yahoo.com/nfl/teams/indianapolis/,['QB'],small,https://s.yimg.com/iu/api/res/1.2/YdWdJ6P7MDCj...,https://s.yimg.com/iu/api/res/1.2/YdWdJ6P7MDCj...,False,False,False,0,Matt,Matt,Matt Hasselbeck,Hasselbeck,4416,331.p.4416,O,QB,https://sports.yahoo.com/nfl/players/4416,2014,3852.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,Matt Hasselbeck,matt hasselbeck,QB,SEA,NaN,NaN,1995.0,6.0,5.0,NaN,NaN,NaN,NaN,California,2025.0,4416.0
8,9,K,nfl.p.4587,Det,Detroit Lions,nfl.t.8,https://sports.yahoo.com/nfl/teams/detroit/,['K'],small,https://s.yimg.com/iu/api/res/1.2/LYV0FawBt9B8...,https://s.yimg.com/iu/api/res/1.2/LYV0FawBt9B8...,False,False,False,0,David,David,David Akers,Akers,4587,331.p.4587,K,K,https://sports.yahoo.com/nfl/players/4587,2014,102.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,David Akers,david akers,PK,PHI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025.0,4587.0
10,4,K,nfl.p.5046,Sea,Seattle Seahawks,nfl.t.26,https://sports.yahoo.com/nfl/teams/seattle/,['K'],small,https://s.yimg.com/iu/api/res/1.2/_FbxgaDsf1YA...,https://s.yimg.com/iu/api/res/1.2/_FbxgaDsf1YA...,False,False,False,0,Sebastian,Sebastian,Sebastian Janikowski,Janikowski,5046,331.p.5046,K,K,https://sports.yahoo.com/nfl/players/5046,2014,4392.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,Sebastian Janikowski,sebastian janikowski,PK,OAK,NaN,NaN,1992.0,7.0,24.0,NaN,NaN,NaN,NaN,Houston,2025.0,5046.0
12,7,QB,nfl.p.5228,TB,Tampa Bay Buccaneers,nfl.t.27,https://sports.yahoo.com/nfl/teams/tampa-bay/,['QB'],small,https://s.yimg.com/iu/api/res/1.2/Ol_o6nNm9irL...,https://s.yimg.com/iu/api/res/1.2/Ol_o6nNm9irL...,False,False,False,0,Tom,Tom,Tom Brady,Brady,5228,331.p.5228,O,QB,https://sports.yahoo.com/nfl/players/5228,2014,948.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tom Brady,tom brady,QB,NEP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025.0,5228.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14336,12,DEF,nfl.p.100028,Was,Washington Commanders,nfl.t.28,https://sports.yahoo.com/nfl/teams/washington/,['DEF'],small,https://s.yimg.com/cv/apiv2/default/nfl/202009...,https://s.yimg.com/cv/apiv2/default/nfl/202009...,False,False,False,0,Washington,Washington,Washington Commanders,Commanders,100028,461.p.100028,DT,DEF,https://sports.yahoo.com/nfl/teams/washington/,2025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

: 

: 

In [ ]:
result[result['name'] == "Travis Hunter"]

,bye_weeks.week,display_position,editorial_player_key,editorial_team_abbr,editorial_team_full_name,editorial_team_key,editorial_team_url,eligible_positions,headshot.size,headshot.url,image_url,is_keeper.status,is_keeper.cost,is_keeper.kept,is_undroppable,name.ascii_first,name.first,name.full,name.last,player_id,player_key,position_type,primary_position,url,season,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season


: 

: 

In [ ]:
# Fill all missing yahoo ids with player IDs

result.loc[(result['yahoo_id'].isna()), 'yahoo_id'] = result.loc[(result['yahoo_id'].isna()), 'player_id']

: 

: 

In [ ]:
result['yahoo_id'].isna().sum()

np.int64(0)

: 

: 

In [ ]:
result[result['name'] == 'Ollie Gordon']

,bye_weeks.week,display_position,editorial_player_key,editorial_team_abbr,editorial_team_full_name,editorial_team_key,editorial_team_url,eligible_positions,headshot.size,headshot.url,image_url,is_keeper.status,is_keeper.cost,is_keeper.kept,is_undroppable,name.ascii_first,name.first,name.full,name.last,player_id,player_key,position_type,primary_position,url,season,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season
13843,12,RB,nfl.p.41969,Mia,Miami Dolphins,nfl.t.15,https://sports.yahoo.com/nfl/teams/miami/,['RB'],small,https://s.yimg.com/iu/api/res/1.2/eDlltI2Ax54R...,https://s.yimg.com/iu/api/res/1.2/eDlltI2Ax54R...,False,False,False,0,Ollie,Ollie,Ollie Gordon II,Gordon II,41969,461.p.41969,O,RB,https://sports.yahoo.com/nfl/players/41969,2025,17059.0,42753bb4-a416-4d40-bf81-f20d2181be2f,25984.0,00-0040198,157792.0,12495.0,58384.0,4711533.0,41969.0,NaN,28873887.0,GordOl00,NaN,18450.0,NaN,1759.0,41969.0,0.0,25953.0,NaN,Ollie Gordon,ollie gordon,RB,MIA,2004-01-15,21.8,2025.0,6.0,3.0,179.0,NaN,74.0,225.0,Oklahoma State,2025.0


: 

: 

In [ ]:
result[(result['gsis_id'].isna()) & (result['primary_position'] != 'DEF') & (result['season'] == 2024)]

,bye_weeks.week,display_position,editorial_player_key,editorial_team_abbr,editorial_team_full_name,editorial_team_key,editorial_team_url,eligible_positions,headshot.size,headshot.url,image_url,is_keeper.status,is_keeper.cost,is_keeper.kept,is_undroppable,name.ascii_first,name.first,name.full,name.last,player_id,player_key,position_type,primary_position,url,season,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season
12125,12,WR,nfl.p.33001,Jax,Jacksonville Jaguars,nfl.t.30,https://sports.yahoo.com/nfl/teams/jacksonville/,['WR'],small,https://s.yimg.com/iu/api/res/1.2/yptHbCexuz6x...,https://s.yimg.com/iu/api/res/1.2/yptHbCexuz6x...,False,False,False,0,Jaylon,Jaylon,Jaylon Moore,Moore,33001,449.p.33001,O,WR,https://sports.yahoo.com/nfl/players/33001,2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33001.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12137,14,TE,nfl.p.33168,Bal,Baltimore Ravens,nfl.t.33,https://sports.yahoo.com/nfl/teams/baltimore/,['TE'],small,https://s.yimg.com/iu/api/res/1.2/NNyZmg0U9BAr...,https://s.yimg.com/iu/api/res/1.2/NNyZmg0U9BAr...,False,False,False,0,Scotty,Scotty,Scotty Washington,Washington,33168,449.p.33168,O,TE,https://sports.yahoo.com/nfl/players/33168,2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33168.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12228,14,TE,nfl.p.33676,Den,Denver Broncos,nfl.t.7,https://sports.yahoo.com/nfl/teams/denver/,['TE'],small,https://s.yimg.com/iu/api/res/1.2/vXbre9RtzGdI...,https://s.yimg.com/iu/api/res/1.2/vXbre9RtzGdI...,False,False,False,0,Hunter,Hunter,Hunter Kampmoyer,Kampmoyer,33676,449.p.33676,O,TE,https://sports.yahoo.com/nfl/players/33676,2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33676.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12231,12,WR,nfl.p.33710,NYJ,New York Jets,nfl.t.20,https://sports.yahoo.com/nfl/teams/ny-jets/,['WR'],small,https://s.yimg.com/iu/api/res/1.2/CmD.PEN8vd8i...,https://s.yimg.com/iu/api/res/1.2/CmD.PEN8vd8i...,False,False,False,0,Brandon,Brandon,Brandon Smith,Smith,33710,449.p.33710,O,WR,https://sports.yahoo.com/nfl/players/33710,2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33710.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12235,11,TE,nfl.p.33759,Ari,Arizona Cardinals,nfl.t.22,https://sports.yahoo.com/nfl/teams/arizona/,['TE'],small,https://s.yimg.com/iu/api/res/1.2/zskwyXeICNy_...,https://s.yimg.com/iu/api/res/1.2/zskwyXeICNy_...,False,False,False,0,Bernhard,Bernhard,Bernhard Seikovits,Seikovits,33759,449.p.33759,O,TE,https://sports.yahoo.com/nfl/players/33759,2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33759.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12816,10,WR,nfl.p.41733,GB,Green Bay Packers,nfl.t.9,https://sports.yahoo.com/nfl/teams/green-bay/,['WR'],small,https://s.yimg.com/iu/api/res/1.2/TcM85WhJ.fAO...,https://s.yimg.com/iu/api/res/1.2/TcM85WhJ.fAO...,False,False,False,0,Rory,Rory,Rory Starkey,Starkey,41733,449.p.41733,O,WR,https://sports.yahoo.com/nfl/players/41733,2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41733.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12817,12,QB,nfl.p.41735,NYJ,New York Jets,nfl.t.20,https://sports.yahoo.com/nfl/teams/ny-jets/,['QB'],small,https://s.yimg.com/iu/api

: 

: 

: 

: 

In [ ]:
result

,bye_weeks.week,display_position,editorial_player_key,editorial_team_abbr,editorial_team_full_name,editorial_team_key,editorial_team_url,eligible_positions,headshot.size,headshot.url,image_url,is_keeper.status,is_keeper.cost,is_keeper.kept,is_undroppable,name.ascii_first,name.first,name.full,name.last,player_id,player_key,position_type,primary_position,url,season,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season
0,10,K,nfl.p.3727,Ind,Indianapolis Colts,nfl.t.11,https://sports.yahoo.com/nfl/teams/indianapolis/,['K'],small,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,False,False,False,0,Adam,Adam,Adam Vinatieri,Vinatieri,3727,331.p.3727,K,K,https://sports.yahoo.com/nfl/players/3727,2014,2842.0,9ecf8040-10f9-4a5c-92da-1b4d77bd6760,9004.0,00-0016919,226.0,120.0,2503471.0,1097.0,3727.0,2074.0,1392.0,vinatada01,NaN,395.0,1152.0,NaN,3727.0,23849.0,3258.0,NaN,Adam Vinatieri,adam vinatieri,PK,FA,1972-12-28,49.2,1996.0,NaN,NaN,NaN,aVinatieri4,72.0,212.0,South Dakota State,2025.0
5912,6,K,nfl.p.3727,Ind,Indianapolis Colts,nfl.t.11,https://sports.yahoo.com/nfl/teams/indianapolis/,['K'],small,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,False,False,False,0,Adam,Adam,Adam Vinatieri,Vinatieri,3727,390.p.3727,K,K,https://sports.yahoo.com/nfl/players/3727,2019,2842.0,9ecf8040-10f9-4a5c-92da-1b4d77bd6760,9004.0,00-0016919,226.0,120.0,2503471.0,1097.0,3727.0,2074.0,1392.0,vinatada01,NaN,395.0,1152.0,NaN,3727.0,23849.0,3258.0,NaN,Adam Vinatieri,adam vinatieri,PK,FA,1972-12-28,49.2,1996.0,NaN,NaN,NaN,aVinatieri4,72.0,212.0,South Dakota State,2025.0
4728,9,K,nfl.p.3727,Ind,Indianapolis Colts,nfl.t.11,https://sports.yahoo.com/nfl/teams/indianapolis/,['K'],small,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,False,False,False,0,Adam,Adam,Adam Vinatieri,Vinatieri,3727,380.p.3727,K,K,https://sports.yahoo.com/nfl/players/3727,2018,2842.0,9ecf8040-10f9-4a5c-92da-1b4d77bd6760,9004.0,00-0016919,226.0,120.0,2503471.0,1097.0,3727.0,2074.0,1392.0,vinatada01,NaN,395.0,1152.0,NaN,3727.0,23849.0,3258.0,NaN,Adam Vinatieri,adam vinatieri,PK,FA,1972-12-28,49.2,1996.0,NaN,NaN,NaN,aVinatieri4,72.0,212.0,South Dakota State,2025.0
7117,7,K,nfl.p.3727,Ind,Indianapolis Colts,nfl.t.11,https://sports.yahoo.com/nfl/teams/indianapolis/,['K'],small,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,False,False,False,0,Adam,Adam,Adam Vinatieri,Vinatieri,3727,399.p.3727,K,K,https://sports.yahoo.com/nfl/players/3727,2020,2842.0,9ecf8040-10f9-4a5c-92da-1b4d77bd6760,9004.0,00-0016919,226.0,120.0,2503471.0,1097.0,3727.0,2074.0,1392.0,vinatada01,NaN,395.0,1152.0,NaN,3727.0,23849.0,3258.0,NaN,Adam Vinatieri,adam vinatieri,PK,FA,1972-12-28,49.2,1996.0,NaN,NaN,NaN,aVinatieri4,72.0,212.0,South Dakota State,2025.0
3523,11,K,nfl.p.3727,Ind,Indianapolis Colts,nfl.t.11,https://sports.yahoo.com/nfl/teams/indianapolis/,['K'],small,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,False,False,False,0,Adam,Adam,Adam Vinatieri,Vinatieri,3727,371.p.3727,K,K,https://sports.yahoo.com/nfl/players/3727,2017,2842.0,9ecf8040-10f9-4a5c-92da-1b4d77bd6760,9004.0,00-0016919,226.0,120.0,2503471.0,1097.0,3727.0,2074.0,1392.0,vinatada01,NaN,395.0,1152.0,NaN,3727.0,23849.0,3258.0,NaN,Adam Vinatieri,adam vinatieri,PK,FA,1972-12-28,49.2,1996.0,NaN,NaN,NaN,aVinatieri4,72.0,212.0,South Dakota State,2025.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

: 

: 

: 

: 